In [5]:
import csv
from functools import reduce
from collections import defaultdict
from statistics import mean
from datetime import datetime

def leer_csv(nombre_archivo):
    try:
        with open(nombre_archivo, mode='r', encoding='utf-8') as archivo:
            lector = csv.DictReader(archivo)
            ventas = []
            for fila in lector:
                ventas.append({
                    'fecha': datetime.strptime(fila['fecha'], '%Y-%m-%d').date(),
                    'producto': fila['producto'],
                    'categoria': fila['categoria'],
                    'cantidad': int(fila['cantidad']),
                    'precio_unitario': float(fila['precio_unitario']),
                })
            return ventas
    except FileNotFoundError:
        print("Error: Archivo no encontrado.")
        return []
    except PermissionError:
        print("Error: Permiso denegado para leer el archivo.")
        return []
    except ValueError:
        print(f"Error: Tipo de dato erroneo encontrado")
        return []
    except Exception as e:
        print(f"Error: Tipo de error desconocido")
        return []

def total_por_categoria(ventas):
    categorias = defaultdict(float)
    list(map(lambda v: categorias.__setitem__(v['categoria'], categorias[v['categoria']] + v['cantidad'] * v['precio_unitario']), ventas))
    return dict(categorias)

def filtrar_ventas_superiores(ventas, umbral):
    return list(filter(lambda v: v['cantidad'] * v['precio_unitario'] > umbral, ventas))

def promedio_ventas_por_dia(ventas):
    ventas_por_dia = defaultdict(list)
    for v in ventas:
        ventas_por_dia[v['fecha']].append(v['cantidad'] * v['precio_unitario'])
    return {dia: mean(valores) for dia, valores in ventas_por_dia.items()}

def producto_mas_vendido(ventas):
    productos = defaultdict(int)
    list(map(lambda v: productos.__setitem__(v['producto'], productos[v['producto']] + v['cantidad']), ventas))
    return max(productos.items(), key=lambda x: x[1])

def ordenar_por_ventas(ventas):
    return sorted(ventas, key=lambda v: v['cantidad'] * v['precio_unitario'], reverse=True)

def resumen_estadistico(ventas):
    montos = list(map(lambda v: v['cantidad'] * v['precio_unitario'], ventas))
    return {
        'total_general': sum(montos),
        'promedio': mean(montos) if montos else 0,
        'transacciones': len(montos)
    }

def identificar_tendencia_diaria(ventas):
    diario = promedio_ventas_por_dia(ventas)
    return sorted(diario.items(), key=lambda x: x[0])

def generar_reporte(ventas, nombre_salida):
    resumen = resumen_estadistico(ventas)
    reporte = [
        f"--- REPORTE DE VENTAS ---",
        f"Total general de ventas: ${resumen['total_general']:.2f}",
        f"Promedio de venta por transacción: ${resumen['promedio']:.2f}",
        f"Número total de transacciones: {resumen['transacciones']}",
        "\nTotal de ventas por categoría:"
    ]
    for cat, total in total_por_categoria(ventas).items():
        reporte.append(f"  {cat}: ${total:.2f}")

    producto, cantidad = producto_mas_vendido(ventas)
    reporte.append(f"\nProducto más vendido: {producto} (Cantidad: {cantidad})")

    reporte.append("\nPromedio de ventas por día:")
    for dia, prom in promedio_ventas_por_dia(ventas).items():
        reporte.append(f"  {dia}: ${prom:.2f}")

    reporte.append("\nVentas con monto superior a $1000:")
    superiores = filtrar_ventas_superiores(ventas, 1000)
    for v in superiores:
        total = v['cantidad'] * v['precio_unitario']
        reporte.append(f"  {v['producto']} ({v['fecha']}): ${total:.2f}")

    with open(nombre_salida, 'w', encoding='utf-8') as f:
        f.write('\n'.join(reporte))
    print(f"Reporte generado en: {nombre_salida}")

if __name__ == '__main__':
    archivo_entrada = 'C:/Users/GOTHICS/OneDrive/Documentos/Jose Cano/1.Emprendimiento/Data Analytics/M3/2.Tarea/ventas_1300.csv'
    archivo_salida = 'C:/Users/GOTHICS/OneDrive/Documentos/Jose Cano/1.Emprendimiento/Data Analytics/M3/2.Tarea/reporte_ventas_1300.txt'
    ventas = leer_csv(archivo_entrada)
    if ventas:
        generar_reporte(ventas, archivo_salida)

Reporte generado en: C:/Users/GOTHICS/OneDrive/Documentos/Jose Cano/1.Emprendimiento/Data Analytics/M3/2.Tarea/reporte_ventas_1300.txt
